In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime

import random
random.seed(42)

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Forex_group' in symbol.path and symbol.spread < 15]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_M30,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['AUDUSDm', 'CADCHFm', 'EURGBPm', 'EURUSDm', 'GBPUSDm', 'NZDCADm', 'TRYDKKm', 'USDCHFm', 'USDJPYm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUDUSDm
AUDUSDm
AUDUSDm
AUDUSDm
AUDUSDm
CADCHFm
CADCHFm
CADCHFm
CADCHFm
CADCHFm
EURGBPm
EURGBPm
EURGBPm
EURGBPm
EURGBPm
EURUSDm
EURUSDm
EURUSDm
EURUSDm
EURUSDm
GBPUSDm
GBPUSDm
GBPUSDm
GBPUSDm
GBPUSDm
NZDCADm
NZDCADm
NZDCADm
NZDCADm
NZDCADm
TRYDKKm
TRYDKKm
TRYDKKm
TRYDKKm
TRYDKKm
USDCHFm
USDCHFm
USDCHFm
USDCHFm
USDCHFm
USDJPYm
USDJPYm
USDJPYm
USDJPYm
USDJPYm


In [5]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{15: Timestamp('2021-10-27 17:15:00'),
 30: Timestamp('2021-10-27 17:15:00'),
 16385: Timestamp('2021-10-27 17:15:00'),
 16386: Timestamp('2021-10-27 17:15:00'),
 16388: Timestamp('2021-10-27 17:15:00')}

In [6]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high

In [7]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
import numpy as np

class MeanReversion(Strategy):
    # Parámetros ajustables de la estrategia
    lookback_period = 50   # Periodo de la SMA
    deviation_threshold = 0.01  # 5% de desviación para generar señal
    risk = 3  # Tamaño del riesgo (posición)
    
    def init(self):
        # Calcular la media móvil simple (SMA)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.lookback_period)
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)

    def next(self):
        # Precio actual y valor de la SMA
        actual_close = self.data.Close[-1]
        sma_value = self.sma[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]

        # Desviación del precio con respecto a la SMA (en porcentaje)
        deviation = (actual_close - sma_value) / sma_value

        if self.position:
            if self.position.is_long and actual_close >= self.sma:
                self.position.close()

            if self.position.is_short and actual_close <= self.sma:
                self.position.close()  
        
        else:
            # Condiciones para comprar (precio por debajo de la SMA más del umbral de desviación)
            if deviation <= -self.deviation_threshold and cum_rsi <= 25:
                self.buy(size=self.risk / 100)

            # Condiciones para vender (precio por encima de la SMA más del umbral de desviación)
            elif deviation >= self.deviation_threshold and cum_rsi >= 75:
                self.sell(size=self.risk / 100)


In [8]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    MeanReversion
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    try:
        start_date = intervals_start_dates[interval]

        bt_train = Backtest(
            symbols[ticker][interval].loc[start_date:], 
            strategy,
            commission=7e-4,
            cash=15_000, 
            margin=1/30
        )
        
        stats = bt_train.run()
        
        equity_curve = stats._equity_curve['Equity'].values    
        x = np.arange(len(equity_curve)).reshape(-1, 1)
        reg = LinearRegression().fit(x, equity_curve)
        stability_ratio = reg.score(x, equity_curve)

        df_stats = pd.DataFrame({
            'strategy':[strategy.__name__],
            'ticker':[ticker],
            'interval':[interval],
            'stability_ratio':[stability_ratio],
            'return':[stats['Return [%]']],
            'final_eq':[stats['Equity Final [$]']],
            'drawdown':[stats['Max. Drawdown [%]']],
            'drawdown_duration':[stats['Max. Drawdown Duration']],
            'win_rate':[stats['Win Rate [%]']], 
            'sharpe_ratio':[stats['Sharpe Ratio']],
            'trades':[stats['# Trades']],
            'avg_trade_percent':[stats['Avg. Trade [%]']],
            'exposure':[stats['Exposure Time [%]']],
            'final_equity':[stats['Equity Final [$]']],
            'Duration':[stats['Duration']],

        })

        performance = pd.concat([performance, df_stats])

    except:
        print(f'hubo un problema con {ticker} {interval}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


In [9]:

filter_performance = performance[
    (performance['return']>0) & (performance['return/dd']>2)
].sort_values(by=['drawdown_duration'], ascending=[True]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,MeanReversion,AUDUSDm,16385,0.821254,13.772442,17065.866374,5.123439,392 days 03:00:00,63.934426,0.747872,122,0.117247,26.342783,17065.866374,1037 days 11:00:00,2.688125,10.823254


In [10]:
for ticker, interval in zip(portfolio, intervals):
    bt_train = Backtest(
        symbols[ticker][interval], 
        MeanReversion,
        commission=7e-4,
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run()

    bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


In [11]:
# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2023, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    mt5.TIMEFRAME_H1

]

tickers = ['AUDUSDm']

symbols = get_data(tickers, intervals, date_from, date_to)
symbols

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUDUSDm


{'AUDUSDm': {16385:                         Open     High      Low    Close  Volume  spread  \
  Date                                                                      
  2023-10-01 21:00:00  0.64185  0.64296  0.64184  0.64294     288      78   
  2023-10-01 22:00:00  0.64310  0.64401  0.64237  0.64278    1630      43   
  2023-10-01 23:00:00  0.64283  0.64367  0.64275  0.64364     725      14   
  2023-10-02 00:00:00  0.64367  0.64446  0.64286  0.64290    1447      11   
  2023-10-02 01:00:00  0.64290  0.64350  0.64268  0.64287    1163      11   
  ...                      ...      ...      ...      ...     ...     ...   
  2024-08-30 16:00:00  0.67644  0.67648  0.67512  0.67578    1982       8   
  2024-08-30 17:00:00  0.67577  0.67617  0.67559  0.67598    1323       8   
  2024-08-30 18:00:00  0.67599  0.67668  0.67592  0.67662    1005       8   
  2024-08-30 19:00:00  0.67661  0.67686  0.67632  0.67671    1159       8   
  2024-08-30 20:00:00  0.67670  0.67703  0.67642  0.67649 

In [12]:
def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

In [14]:
from utils import walk_forward

lookback_bars = 1920
validation_bars = 100
warmup_bars = 80

params = {
    'deviation_threshold': list(np.arange(0.01, 0.1, 0.01)),
    'lookback_period': [12, 26, 50],
    'maximize': optim_func
}

stats = walk_forward(
    MeanReversion,
    symbols['AUDUSDm'][16385], 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 15000.0


  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 15000.0


  0%|          | 0/9 [00:00<?, ?it/s]

...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


equity final: 15523.204250214012


  0%|          | 0/9 [00:00<?, ?it/s]

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


equity final: 15496.728355432013


In [16]:
from utils import plot_full_equity_curve

plot_full_equity_curve(
    symbols['AUDUSDm'][16385], 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True
)

In [17]:
from utils import plot_stats

for x in range(0, len(stats)):
    plot_stats(symbols['AUDUSDm'][16385], stats[x], MeanReversion, plot=True)

Start                     2024-01-18 14:00:00
End                       2024-01-30 01:00:00
Duration                     11 days 11:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      15000.0
Equity Peak [$]                       15000.0
Return [%]                                0.0
Buy & Hold Return [%]                0.880002
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2024-01-24 18:00:00
End                       2024-02-05 05:00:00
Duration                     11 days 11:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      15000.0
Equity Peak [$]                       15000.0
Return [%]                                0.0
Buy & Hold Return [%]               -1.109593
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2024-01-30 22:00:00
End                       2024-02-09 09:00:00
Duration                      9 days 11:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      15000.0
Equity Peak [$]                       15000.0
Return [%]                                0.0
Buy & Hold Return [%]               -1.424135
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2024-08-09 09:00:00
End                       2024-08-20 20:00:00
Duration                     11 days 11:00:00
Exposure Time [%]                        15.0
Equity Final [$]                  15523.20425
Equity Peak [$]                   15584.03077
Return [%]                          -0.280283
Buy & Hold Return [%]                2.450027
Return (Ann.) [%]                    -6.82871
Volatility (Ann.) [%]                0.948658
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.390313
Avg. Drawdown [%]                   -0.167327
Max. Drawdown Duration        0 days 15:00:00
Avg. Drawdown Duration        0 days 09:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.311438
Worst Trade [%]                     -0.311438
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2024-08-15 13:00:00
End                       2024-08-27 00:00:00
Duration                     11 days 11:00:00
Exposure Time [%]                   31.666667
Equity Final [$]                 15496.728355
Equity Peak [$]                  15540.351655
Return [%]                          -0.170557
Buy & Hold Return [%]                2.180318
Return (Ann.) [%]                   -3.835161
Volatility (Ann.) [%]                1.240087
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.457156
Avg. Drawdown [%]                   -0.189609
Max. Drawdown Duration        6 days 19:00:00
Avg. Drawdown Duration        2 days 10:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.189512
Worst Trade [%]                     -0.189512
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

In [18]:
len(stats)

37